In [1]:
import numpy as np
import math
from functools import *

![Question 16](./images/q16.png)

在课堂上, 我们讲授了一维数据的'正负射线'(简单来说就是一维感知器)的学习模型. 该模型包含以下形式的假设:

$$
h_{s,\theta}(x) = s \cdot{} sign(x - \theta)
$$

该模型通常被称为'决策树桩'模型, 是最简单的学习模型之一. 如课堂所示, 对于一维数据, 决策树桩模型的 VC 维为 2.

事实上, 决策树桩模型是我们可以​通过枚举所有可能的阈值来有效地轻松最小化 $E_{in}$ 的少数模型之一. 特别地, 对于 $N$ 个例子, 最多有 $ 2N $ 个 dichotomy(参见第 5 课幻灯片的第 22 页), 因此最多有 $2N$ 个不同 $E_{in}$ 值. 然后我们可以轻松地选择使得 $E_{in}$ 最小的 dichotomy, 其中可以通过在最小的 $E_{in}$ 中随机选择来消除平局(原文: We can then easily choose the dichotomy that leads to the lowest $E_{in}$, where ties can be broken by randomly choosing among the lowest $E_{in}$ ones). 所选的 dichotomy 表示某些点($\theta$ 范围)和 $s$ 的组合(这里是说 $\theta$ 和 $s$ 的组合确定了一个 dichotomy, 对这个 dichotomy 来说, $\theta$ 的取值本身是一个范围, 在这个范围内, 给定的样本都可以形成同一个 dichotomy), 通常将该范围的中值选作实现 dichotomy 的 $\theta$.

在本题中, 我们将实现这样的算法, 并在人工数据集上运行程序.

首先, 通过以下过程生成一维数据:

1. 在 $[-1,1]$ 中通过均匀分布生成 $x$
2. 通过 $f(x) = \tilde{s}(x) + noise$ 生成 $y$, 其中 $\tilde{s}(x) = sign(x)$ 并且噪声以 20% 的概率翻转结果.

In [2]:
def h(x, s, theta):
    t = x - theta
    return s * (-1 if t < 0 else 1)

In [3]:
def example_data(size):
    """
    f(x) = sign(x), flips result with 20% probability
    """

    def sign(data):
        return [1 if x > 0 else -1 for x in data]

    def flip(x):
        if np.random.rand() < 0.2:
            return -x

        return x

    x = np.random.uniform(-1, 1, size)
    y = [flip(a) for a in sign(x)]

    res = list(zip(x, y))
    # np.random.shuffle(res)

    return res

## Question 16

对于任何决策树桩 $h_{s,\theta}$, $\theta \in [-1,1]$, 将 $E_{out}(h_{s,\theta})$ 表示为 $\theta$ 和 $s$ 的函数.

**解:**

考虑作业 2 的第一题, 提到有噪声版的目标函数 $f$ 用下面的形式给出:

$$
P(y \mid x) = \begin{cases}
      \lambda & y = f(x)\\
      1 - \lambda & \text{otherwise}
    \end{cases}      
$$


假设函数 $h$ 犯错误的概率是 $ \mu $, 那么错误概率可以写成:

$$
E_{out} = \lambda \mu + (1-\lambda)(1-\mu)
$$

> $E_{in}, E_{out}$ 可以用这种概率形式表示吗?
>
> 在未来课程中, 逻辑斯蒂回归使用的交叉熵错误, 看上去是一个概率形式的表达.

题目中, 已经告诉我们 $\lambda = 1-20\% = 0.8$, 下面试着计算 $\mu$, 情况比较复杂, 对 $s$ 分类讨论(决策树桩的 $s \in \{-1, +1\} $):

1. $s = +1$, $\theta > 0$ 时, $x \in [-1, 1]$ 的点里面, $x \in [0, \theta]$ 部分是分类错误(和无噪声的 $f$ 不一样)的, 因此有 $\mu = \frac{\theta}{2}$
2. $s = +1$, $\theta < 0$ 时, 参考 1 的分析, 有 $\mu = \frac{|\theta|}{2}$
3. $s = -1$, $\theta > 0$ 时, 参考 1 的分析, 有 $\mu = 1 - \frac{\theta}{2}$
4. $s = -1$, $\theta < 0$ 时, 参考 1 的分析, 有 $\mu = 1 - \frac{|\theta|}{2}$

把上面四种情况写在一起(注意这不是对概率在求和, 只是将上面四种情况用一个式子表达了出来而已):

$$
\begin{align*}
\mu &= \frac{1+s}{2} \times \frac{|\theta|}{2} + \frac{1-s}{2} \times (1 - \frac{|\theta|}{2}) \\
&= \frac{1}{2}(s |\theta| - s + 1)
\end{align*}
$$

$E_{out}$ 里面代入 $\lambda$ 和 $\mu$, 可以得到 $E_{out} = 0.5 + 0.3s(|\theta| - 1)$

![Question 17](./images/q17.jpg)

## Question 17

按照上述过程生成大小为 20 的数据集, 并在数据集上运行一维决策树桩算法. 记录 $E_{in}$ 并使用上述公式计算 $E_{out}$, 重复该实验(包括数据生成, 运行决策树桩算法以及计算 $E_{in}$ 和 $E_{out}$) 5000 次. $E_{in}$ 的平均值是多少?

**解:**

按照题目的说法, 我们要暴力遍历所有的 dichotomy, 因此先准备好能决定 dichotomy 的 $(\theta, s)$ 集合. 如果 $x \in \{ x_1, x_2, ... x_{20} \mid x_1 < x_2 < ... < x_{20} \}$, 那么 $\theta$ 可以取的值有(使用每个 dichotomy $\theta$ 范围的中值) $ \frac{-1 + x_1}{2}, \frac{x_1 + x_2}{2}, ..., \frac{x_{19} + x_{20}}{2}, \frac{x_{20} + 1}{2} $, 共计 21 个点.

至于 $s$, 它仅可以取 $\{-1, +1\}$.

> 出于计算后面的 19/20 题通用考虑, 下面的代码里面对第一个和最后一个 $\theta$ 的处理, 不是用的 $-1$, $+1$ 的边界, 而是用的 $ x_0 - 1$ 和 $ x_20 + 1 $.

In [4]:
def get_theta_list(data):
    result = []

    xs = sorted([x[0] for x in data])

    prev = xs[0] - 1
    for x in xs:
        result.append((prev + x) / 2)
        prev = x

    result.append((prev + prev + 1) / 2)
    
    return result

def get_dichotomy_theta_and_s(data):
    result = []

    theta_list = get_theta_list(data)
    
    for theta in theta_list:
        result.append((theta, -1))
        result.append((theta, +1))

    return result

有了 $(\theta, s)$ 的集合之后, 所要做的事情就是遍历这个集合, 并计算在样本点上的 $E_{in}$, 然后用上面的公式可以算 $E_{out}$ 出来.

In [5]:
def decision_stump(data):
    dichotomy_list = get_dichotomy_theta_and_s(data)

    best = (0, 0)
    best_in = np.inf
    best_out = np.inf
    
    for theta, s in dichotomy_list:
        _h = partial(h, s=s, theta=theta)
        
        # 代入数据运算
        err = 0
        for x, y in data:
            err += _h(x) == y

        if err < best_in:
            best_in = err
            best = (theta, s)

        err_out = 0.5 + 0.3*s*(np.abs(theta) - 1)
        if err_out < best_out:
            best_out = err_out
            # print(s, theta, err_out)

    return best_in / len(data), best_out, best

In [6]:
loops = 5000

err_in = 0
err_out = 0

for i in range(loops):
    data = example_data(size=20)
    e_in, e_out, _ = decision_stump(data)
    
    err_in += e_in
    err_out += e_out

print("err_in", err_in / loops)
print("err_out", err_out / loops)

err_in 0.16993999999999992
err_out 0.21064865779191114


![Question 18](./images/q18.png)

### Question 18

**解:**

参考 Question 17 里面关于 $E_{out}$ 的计算.

---

我最开始的更新错误部分的写法实际上是:


```python
if err < best_in:
    best_in = err
    best_out = 0.5 + 0.3*s*(np.abs(theta) - 1)
```


也就是只有在遇到更好的 err_in 的时候, 才用它的 $s, \theta$ 来更新一下 `best_out`, 但是这样算出来 $E_{out}$ 均值达到了 $0.7$


> **!!!**
>
> 这里为什么要在无论何种场景下都要更新 $E_{out}$ 呢?

![Question 19](./images/q19.png)

### Question 19

决策树桩也适用于多维数据. 特别地, 现在每个决策树桩处理一个特定的维度 $i$, 如下:

$$
h_{s,i,\theta}(x) = s \cdot{} sign(x_i - \theta)
$$

对多维数据实现以下决策树桩算法:

1. 对于每个维度 $i = 1, 2, ... , d$, 使用你刚刚实现的一维决策桩算法找到最佳决策桩 $h_{s,i, \theta}$
2. 返回以 $E_{in}$ 而言 '最佳中的最佳' 决策桩. 如果出现平局, 请从 $E_{in}$ 最小的桩中随机选择一个(原文: return the "best of best"' decision stump in terms of $E_{in}$. If there is a tie , please randomly choose among the lowest-$E_{in}$ ones, 原文读的有点稀里糊涂, 这意思是不是说计算好 N 个维度之后, 返回 N 个维度最小的 $E_{in}$ 里面的那个最小的? 从网上别人的代码看, 好像是要这么干)

训练数据 $\mathcal{D}_{train}$ 可在以下位置获得: [https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw2_train.dat](https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw2_train.dat)

测试数据 $\mathcal{D}_{test}$ 可在以下位置获得: [https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw2_test.dat](https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw2_test.dat)

对 $\mathcal{D}_{train}$ 运行该算法, 报告您的程序返回的最佳决策桩的 $E_{in}$.

**解:**

读取并处理原始数据, 然后送到 Question 17 实现的算法里面计算即可.

In [7]:
def read_data(file):
    result = []

    fh = open(file)
    for line in fh.readlines():
        parts = line.strip().split(" ")

        x = [float(x) for x in parts[:-2]]
        y = int(parts[-1])

        result.append((x, y))

    return result

def train(_data):
    dim = len(_data[0][0])

    params = []
    best_in = np.inf
    
    for i in range(dim):
        data = [(x[i], y) for x, y in _data]
        err_in, err_out, best = decision_stump(data)
        
        # 记一下这维度上表现最好的 theta 和 s, 以后预测可以用
        params.append(best)

        if err_in < best_in:
            best_in = err_in

    return best_in, params

train_dat = read_data("hw2_train.dat")
err_in, params = train(train_dat)

print(err_in)

0.25


![Question 20](./images/q20.png)

### Question 20

用第 19 题求得的 params, 在 $\mathcal{D}_{test}$ 上面执行预测, 并收集每个维度上的错误情况, 最后报告一个最小的错误.

In [8]:
def test(params):
    _data = read_data("hw2_test.dat")

    err_out = []
    for i, (theta, s) in enumerate(params):
        _h = partial(h, s=s, theta=theta)
        
        # 预测测试集数据
        data = [(x[i], y) for x, y in _data]

        err = 0
        for x, y in data:
            err += _h(x) == y

        err_out.append(err)

    return np.array(err_out) / len(_data)

res = test(params)
np.min(res)

0.355